# Model PEFT training

In [3]:
!pip install evaluate
!pip install jiwer
!pip install IProgress
!pip install bitsandbytes

# !pip install transformers[torch]
# !pip install soundfile
# !pip install torchaudio

## Load Data

In [4]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..')))

from modules.data import load_quebecois_data

data = load_quebecois_data(save=True)

/home/ulaval.ca/mapod13/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 5389
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 1348
    })
})


Saving the dataset (4/4 shards): 100%|██████████| 1348/1348 [00:08<00:00, 167.34 examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'input_features', 'labels'],
        num_rows: 5389
    })
    test: Dataset({
        features: ['audio', 'text', 'input_features', 'labels'],
        num_rows: 1348
    })
})


## Create LoRA model

In [6]:
from peft import get_peft_model, LoraConfig

from modules.model import get_whisper

model_language = "french"
whisper_model = get_whisper()

whisper_model.generation_config.language = model_language
whisper_model.generation_config.task = "transcribe"
#Freeze Whisper weights
whisper_model.requires_grad_(False)

peft_config = LoraConfig(r=32,
                         lora_alpha=64,
                         target_modules=['q_proj', 'v_proj'],
                         bias='none',
                         use_dora=True,
                         modules_to_save=["embed_tokens", "lm_head"])

lora_model = get_peft_model(whisper_model, peft_config)
lora_model.print_trainable_parameters()


NameError: name 'LoraConfig' is not defined

## Setup trainer

In [1]:
from pathlib import Path
from modules.training import compute_metrics, DataCollatorSpeechSeq2SeqWithPadding
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, WhisperProcessor

model_name = "openai/whisper-small"
tokenizer_language = "French"
processor = WhisperProcessor.from_pretrained(model_name, language="French", task="transcribe")

training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    gradient_checkpointing=True,
    learning_rate=1e-5,
    max_steps=7000,
    warmup_steps=500,
    save_strategy='best',  #Keep only best model when saving
    save_steps=5,
    save_only_model=True,
    save_total_limit=2,
    # eval_strategy="no",
    eval_strategy="steps",
    eval_steps=5,
    fp16=True,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=100,
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    optim="adamw_bnb_8bit"
)
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor,
                                                     decoder_start_token_id=whisper_model.config.decoder_start_token_id)
print(data)
print(data['test'].features)
test = data["test"].take(5)
lora_model.config.use_cache = False
lora_model.enable_input_require_grads()
# lora_model.enable_input_require_grads()
trainer = Seq2SeqTrainer(
    args=training_args,
    model=lora_model,
    train_dataset=data["train"],
    eval_dataset=test,
    data_collator=data_collator,
    processing_class=processor,
    compute_metrics=compute_metrics,
)
print(trainer)

ModuleNotFoundError: No module named 'modules'

## Training

In [2]:
from datetime import datetime

#Create new directory for checkpoints with the model languages
output_dir = Path(
    f'cps/cp_{model_name.replace('/', '-')}-{model_language}_T{tokenizer_language}_{datetime.now().strftime("%d-%m-%Y_%H:%M")}')
output_dir.mkdir(parents=True, exist_ok=True)
trainer.args.output_dir = str(output_dir)
trainer.train()

NameError: name 'model_name' is not defined

In [7]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import PeftModel

# 1. Define paths
base_model_name = "openai/whisper-small"
local_adapter_path = "./notebooks/cps/cp_openai-whisper-small-french_TFrench_21-12-2025_11:46/checkpoint-1000"  # Folder containing adapter_model.bin/safetensors

# 2. Load the base model and processor
model = WhisperForConditionalGeneration.from_pretrained(base_model_name)
processor = WhisperProcessor.from_pretrained(base_model_name)

# 3. Load the PEFT adapter from your local directory
model = PeftModel.from_pretrained(model, local_adapter_path)

# (Optional) Move to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(model)

ValueError: Can't find 'adapter_config.json' at './notebooks/cps/cp_openai-whisper-small-french_TFrench_21-12-2025_11:46/checkpoint-1000'